# Pre set up

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# install dependencies: 
!pip install pyyaml==5.1 pycocotools>=2.0.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

# install detectron2: (Colab has CUDA 10.1 + torch 1.6)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
assert torch.__version__.startswith("1.6")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html

1.6.0+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html
     |████████████████████████████████| 6.6MB 621kB/s 
     |████████████████████████████████| 2.2MB 12.2MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.2.post20201030-cp36-none-any.whl size=44294 sha256=9d4ab4ffbb90c21bc232305d3ffb55efa21dae22800c2bf58eec71a6f3a0cc43
  Stored in directory: /root/.cache/pip/wheels/7f/7a/c8/521a7c1d1974bf1c536fed7cb258e3be173730d71065407bab
Successfully built fvcore
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0


In [1]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os
import cv2
import random
# from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog,MetadataCatalog
from detectron2.engine import DefaultTrainer
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 10,10
def imshow(img):
    plt.imshow(img[:, :, [2, 1, 0]])
    plt.axis("off")
    plt.show()

# Train ORCNN on amodal datasets

## Register my amodal datasets 

In [2]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog
dataDir='/content/drive/My Drive/CS DOP/mechanics_dataset_2'
annTrainFile='{}/train/coco_instances.json'.format(dataDir)
imgTrainFile = '{}/train/images'.format(dataDir)
register_coco_instances("amodal_coco_train", {},annTrainFile , imgTrainFile)

# Prepare test datasets
annTestFile='{}/val/coco_instances.json'.format(dataDir)
imgTestFile = '{}/val/images'.format(dataDir)
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog
register_coco_instances("amodal_coco_val", {}, annTestFile, imgTestFile)

In [17]:
dataset_dicts = DatasetCatalog.get("amodal_coco_train")
annos = [img['annotations'] for img in dataset_dicts ]
# invisible_mask = [ obj['invisible_mask'] for img in annos for obj in img if obj.get('invisible_mask')]
annos[0][0].keys()
# visible_mask = [ obj['visible_mask'] for img in annos for obj in img ]
# visible_mask = [ obj['segmentation'] for img in annos for obj in img ]
# print(invisible_mask[2])
# [len(i) for i in invisible_mask]

[10/31 08:28:48 d2.data.datasets.coco]: Loaded 10000 images in COCO format from /content/drive/My Drive/CS DOP/mechanics_dataset_2/train/coco_instances.json


dict_keys(['iscrowd', 'bbox', 'category_id', 'segmentation', 'bbox_mode'])

## Sanity checks : overfitting small datasets 

In [ ]:
# Register small datasets for debugging 
dataset_dicts = DatasetCatalog.get("amodal_coco_train")
import random
imgs = random.sample(dataset_dicts,k=10)
# imgs = dataset_dicts[:10]
DatasetCatalog.register("small_amodal_test", lambda : imgs)
metadata = {}
MetadataCatalog.get("small_amodal_test").set(
        image_root="datasets/coco/train2014", evaluator_type="coco", **metadata
    )


In [ ]:
from pycocotools.coco import COCO
coco_api = COCO(annTrainFile)
cat_ids = sorted(coco_api.getCatIds())
cats = coco_api.loadCats(cat_ids)
# The categories in a custom json file may not be sorted.
thing_classes = [c["name"] for c in sorted(cats, key=lambda x: x["id"])]

In [ ]:
metadata = {}
MetadataCatalog.get("small_amodal_test").set(thing_classes = thing_classes, **metadata )
# MetadataCatalog.get("small_amodal_test")

In [ ]:
smalldicts = DatasetCatalog.get("small_amodal_test")
for i in range(len(smalldicts)):
    im = cv2.imread(smalldicts[i]["file_name"])
    imshow(im[:, :, ::-1])
    visualizer = Visualizer(im[:, :, ::-1], metadata=MetadataCatalog.get("small_amodal_test"), scale=0.5)
    vis = visualizer.draw_dataset_dict(smalldicts[i])
    imshow(vis.get_image()[:, :, ::-1])

### Training 
- remove orcnnEvaluation directory everytime : create json_file for small dict

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_orcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("small_amodal_test",)
cfg.DATASETS.TEST = ("small_amodal_test",)
cfg.DATALOADER.NUM_WORKERS = 2
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0015  # pick a good LR
# cfg.SOLVER.STEPS = (1300,1400)
cfg.SOLVER.MAX_ITER = 1000
cfg.VIS_PERIOD = 20
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 80
cfg.OUTPUT_DIR = "orcnnCheckpoint"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9  # set the testing threshold for this model
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
# import pdb; pdb.set_trace()
trainer.train()

### Tensorboard

In [ ]:
!tensorboard --logdir=orcnnCheckpoint --port=6006

### Evaluation

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9   # set the testing threshold for this model
#  evaluate its performance using AP metric implemented in COCO API.
from detectron2.evaluation import AmodalEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = AmodalEvaluator("small_amodal_test", cfg, False, output_dir="orcnnEvaluation")
val_loader = build_detection_test_loader(cfg, "small_amodal_test")
# import pdb;pdb.set_trace()
inference_on_dataset(trainer.model, val_loader, evaluator) 

### Visualization

In [ ]:
from detectron2.utils.visualizer import ColorMode
import random
from detectron2.data import DatasetCatalog
from detectron2.utils.visualizer import Visualizer
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9   # set the testing threshold for this model
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = ("small_amodal_test",)
predictor = DefaultPredictor(cfg)
dataset_dicts = DatasetCatalog.get("small_amodal_test")
# for d in random.sample(dataset_dicts, 2): 
for i,d in enumerate(dataset_dicts):    
    im = cv2.imread(d["file_name"])
    visualizer = Visualizer(im[:, :, ::-1], metadata=MetadataCatalog.get("small_amodal_test"), scale=0.5)
    vis = visualizer.draw_dataset_dict(d,'invisible_mask')
    imshow(vis.get_image()[:, :, ::-1])
#     import pdb;pdb.set_trace()
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=MetadataCatalog.get("small_amodal_test"), 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"),'pred_invisible_masks')
    imshow(v.get_image()[:, :, ::-1])
    cv2.imwrite('SmallSamplesVisualizing/Sample{}.png'.format(i), vis.get_image()[:, :, ::-1]) 
    cv2.imwrite('SmallSamplesVisualizing/Test{}.png'.format(i), v.get_image()[:, :, ::-1]) 

## Training from a COCO-pretrained model as provided by Detectron2
1. Finetuning : For the case of COCOA amodal the final output layers that are class-specific had to be initialized randomly as the number of classes and their semantic meaning did not fit to the number of classes of COCO.

## Training 

In [ ]:
import os
import numpy as np
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_orcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("amodal_coco_train",)
cfg.DATASETS.TEST = ("amodal_coco_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 80
cfg.SOLVER.BASE_LR = 0.0005  # pick a good LR
cfg.SOLVER.STEPS = (6000,8000)
cfg.SOLVER.MAX_ITER = 10000 
cfg.VIS_PERIOD = 500
cfg.OUTPUT_DIR = "orcnnCheckpoint"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9  # set the testing threshold for this model
import pdb; pdb.set_trace()
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
!tensorboard --logdir=orcnnCheckpoint --port=6006

## Evalution

In [ ]:
cfg.DATASETS.TEST = ("amodal_coco_val",)
cfg.DATASETS.TRAIN = ("amodal_coco_train",)
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9   # set the testing threshold for this model
#  evaluate its performance using AP metric implemented in COCO API.
from detectron2.evaluation import AmodalEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = AmodalEvaluator("amodal_coco_val", cfg, False, output_dir="orcnnEvaluation")
val_loader = build_detection_test_loader(cfg, "amodal_coco_val")
inference_on_dataset(trainer.model, val_loader, evaluator)

## Visualization

In [ ]:
from detectron2.utils.visualizer import ColorMode
import random
from detectron2.data import DatasetCatalog
from detectron2.utils.visualizer import Visualizer
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_orcnn_R_50_FPN_3x.yaml"))
cfg.OUTPUT_DIR = "orcnnCheckpoint"
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 80
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9   # set the testing threshold for this model
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = ("amodal_coco_val",)
predictor = DefaultPredictor(cfg)
dataset_dicts = DatasetCatalog.get("amodal_coco_val")
for d in random.sample(dataset_dicts, 1):    
    im = cv2.imread(d["file_name"])
    visualizer = Visualizer(im[:, :, ::-1], metadata=MetadataCatalog.get("amodal_coco_val"), scale=0.5)
    vis = visualizer.draw_dataset_dict(d,"invisible_mask")
    imshow(vis.get_image()[:, :, ::-1])
#     import pdb;pdb.set_trace()
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=MetadataCatalog.get("amodal_coco_val"), 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"),'pred_invisible_masks')
    imshow(v.get_image()[:, :, ::-1])

In [ ]:
im = cv2.imread(d["file_name"])
visualizer = Visualizer(im[:, :, ::-1], metadata=MetadataCatalog.get("amodal_coco_val"), scale=0.5)
vis = visualizer.draw_dataset_dict(d,"invisible_mask")
imshow(vis.get_image()[:, :, ::-1])
#     import pdb;pdb.set_trace()
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
           metadata=MetadataCatalog.get("amodal_coco_val"), 
           scale=0.8, 
           instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"),'pred_invisible_masks')
imshow(v.get_image()[:, :, ::-1])